# **Segmentação de MRI Cardíacas**



# Dependências

Bibliotecas

In [ ]:
!pip install tensorflow==2.5.0
!pip install scikit-learn==0.24.2
!pip install focal-loss==0.0.7

!pip install nibabel==3.2.1
!pip install MedPy==0.4.0

!pip install matplotlib==3.2.2

Acesso à arquivos do Gdrive

In [ ]:
import os, sys
from google.colab import drive

drive_path = [
    '/content/drive',
    'My Drive/Colab Notebooks/data'
]

drive.mount(drive_path[0], force_remount=True)
os.chdir('/'.join(drive_path))
sys.path.append('/'.join(drive_path))

Visualização da GPU disponibilizada

In [ ]:
!nvidia-smi

Visualizando a quantidade de GPUs detectadas

In [ ]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
print(f'GPUs available: {len(gpus)}')

Configuração de uso gradual e necessário da memória da GPU utilizada

In [ ]:
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f'{len(gpus)} physical GPUs vs. {len(logical_gpus)} logical GPUs')
        
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

# Constantes

In [ ]:
IMG_SZ = 200

In [ ]:
RANDOM_SEED = 42

Dados

In [ ]:
PATCH_STEP = 50

In [ ]:
DT_BUFFER_SZ = 150

In [ ]:
VALIDATION_PROPORTION = .2

In [ ]:
XVALIDATION_PROPORTION = .65

In [ ]:
# represent voxels located in the:
# 0: background
# 1: RV cavity
# 2: myocardium
# 3: LV cavity 
CLASSES_CNT = 4

In [ ]:
TRAINING_FILENAME = 'training.zip'
TRAINING_DIR = 'ACDC/train'

TRAINING_FILENAME = os.path.join(*drive_path, TRAINING_FILENAME)
TRAINING_DIR = os.path.join(*drive_path, TRAINING_DIR)

Modelo

In [ ]:
MODEL_FILENAME = 'ulas_model'

In [ ]:
MODEL_DIR = os.path.join(*drive_path, MODEL_FILENAME)

Métricas e Funções de perda

In [ ]:
TVERSKY_FN_WEIGHT = .7

In [ ]:
TVERSKY_FOCAL = .5

In [ ]:
FOCAL_LOSS_GAMA = 2

Treinamento

In [ ]:
INIT_LEARNING_RATE = 1e-4

In [ ]:
MIN_LEARNING_RATE = 1e-6

In [ ]:
BATCH_SIZE = 4

In [ ]:
MAIN_METRIC = 'dice_index_m'

In [ ]:
if MODEL_FILENAME == 'unet_plusplus_model':
    EVAL_METRIC = f're_lu_{MAIN_METRIC}'
else:
    EVAL_METRIC = MAIN_METRIC

Milestones do treinamento

In [ ]:
EARLY_STOP_PATIENCE = 5

In [ ]:
STEPS_FILENAME = os.path.join(
    MODEL_DIR,
    '{}_improvement_{}.h5'.format(
        MODEL_FILENAME,
        '{}_{}'.format(
            '{epoch:02d}',
            ''.join(['{', f'val_{EVAL_METRIC}:.2f', '}'])
        )
    )
)

In [ ]:
CSV_LOG_FILENAME = os.path.join(MODEL_DIR, f'{MODEL_FILENAME}_log.csv')

Resultados

In [ ]:
PLOT_FILENAME = f'{MODEL_FILENAME}_structure.png'
PLOT_FILENAME = os.path.join(MODEL_DIR, PLOT_FILENAME)

In [ ]:
PLOT_WIDTH = 720

# Aquisição das imagens

Imports desta seção

In [ ]:
import os

import nibabel as nib
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.image import extract_patches
from tensorflow.keras.utils import normalize, to_categorical

In [ ]:
"""
author: Clément Zotti (clement.zotti@usherbrooke.ca)
date: April 2017

DESCRIPTION :
The script provide helpers functions to handle nifti image format:
    - load_nii()
    - save_nii()

to generate metrics for two images:
    - metrics()

And it is callable from the command line (see below).
Each function provided in this script has comments to understand
how they works.

HOW-TO:

This script was tested for python 3.4.

First, you need to install the required packages with
    pip install -r requirements.txt

After the installation, you have two ways of running this script:
    1) python metrics.py ground_truth/patient001_ED.nii.gz prediction/patient001_ED.nii.gz
    2) python metrics.py ground_truth/ prediction/

The first option will print in the console the dice and volume of each class for the given image.
The second option wiil ouput a csv file where each images will have the dice and volume of each class.


Based on: http://acdc.creatis.insa-lyon.fr
"""
#
# Utils function to load and save nifti files with the nibabel package
#
def load_nii(img_path):
    """
    Function to load a 'nii' or 'nii.gz' file, The function returns
    everyting needed to save another 'nii' or 'nii.gz'
    in the same dimensional space, i.e. the affine matrix and the header

    Parameters
    ----------

    img_path: string
    String with the path of the 'nii' or 'nii.gz' image file name.

    Returns
    -------
    Three element, the first is a numpy array of the image values,
    the second is the affine transformation of the image, and the
    last one is the header of the image.
    """
    nimg = nib.load(img_path)
    return nimg.get_fdata().astype('float32'), nimg.affine, nimg.header

def save_nii(img_path, data, affine, header):
    """
    Function to save a 'nii' or 'nii.gz' file.

    Parameters
    ----------

    img_path: string
    Path to save the image should be ending with '.nii' or '.nii.gz'.

    data: np.array
    Numpy array of the image data.

    affine: list of list or np.array
    The affine transformation to save with the image.

    header: nib.Nifti1Header
    The header that define everything about the data
    (pleasecheck nibabel documentation).
    """
    nimg = nib.Nifti1Image(data, affine=affine, header=header)
    nimg.to_filename(img_path)

In [ ]:
def file_paths(base_dir):
    c_paths = {
        'config': [],
        'ground_truth': [],
        'images': [],
        '4d': []
    }

    for c_dir, next_dirs, file_names in os.walk(base_dir):
        # skip root
        if next_dirs:
            continue

        c_patient = os.path.basename(c_dir)

        for f_n in file_names:
            f_n = os.path.join(c_dir, f_n)

            if f_n.endswith('.cfg'): file_type = 'config'
            elif '_gt' in f_n: file_type = 'ground_truth'
            elif '_4d' in f_n: file_type = '4d'
            else: file_type = 'images'

            c_paths[file_type].append(f_n)

    for file_type in c_paths.keys():
        c_paths[file_type].sort()

    return c_paths

def load_data(file_paths, is_training=True):
    images_filename = os.path.join(*drive_path, f'{"training" if is_training else "test"}_data.npy')
    gt_filename = os.path.join(*drive_path, f'{"training" if is_training else "test"}_gt_data.npy')

    try:
        imgs = np.load(images_filename, allow_pickle=True)
        imgs_gt = np.load(gt_filename, allow_pickle=True)

        if np.any(imgs) and np.any(imgs_gt):
            print('carregado dos arquivos')

            return tf.convert_to_tensor(imgs), tf.convert_to_tensor(imgs_gt)
    except:
        print('carregando arquivos')
        imgs, imgs_gt =  _load_data(file_paths)

        # np.save(images_filename, imgs)
        # np.save(gt_filename, imgs_gt)

        return imgs, imgs_gt

def _load_data(file_paths):
    images, ground_truth = [], []
    cnt = 1

    for c_img, c_ground_truth in zip(file_paths['images'], file_paths['ground_truth']):
        print(f'processing {cnt}...')

        # we load 3D training image
        training_image, _, _ = load_nii(c_img)
        # we load 3D training mask (shape=(512,512,129))
        train_ground_truth, _, _ = load_nii(c_ground_truth)

        for k in range(min(train_ground_truth.shape[-1], training_image.shape[-1])):
            #axial cuts are made along the z axis with undersampling
            gt_2d = np.array(train_ground_truth[::, ::, k])

            # invalid ground truth
            if len(np.unique(gt_2d)) == 1:
                continue

            image_patches = _extract_patches(_pre_process_img(np.array(training_image[::, ::, k])))
            gt_patches = _extract_patches(gt_2d)

            if (tf.size(images) == 0).numpy():
                images = image_patches
            else:
                images = tf.concat((images, image_patches), axis=0)

            if (tf.size(ground_truth) == 0).numpy():
                ground_truth = gt_patches
            else:
                ground_truth = tf.concat((ground_truth, gt_patches), axis=0)

        if cnt >= 50:
            break

        cnt += 1

    # images = normalize(images)
    ground_truth = tf.cast(ground_truth, tf.int32)

    return images, ground_truth

def _extract_patches(image_2d):
    image_2d = tf.expand_dims(tf.expand_dims(image_2d, axis=-1), axis=0)

    if image_2d.shape[1] < IMG_SZ or image_2d.shape[2] < IMG_SZ:
        return tf.image.resize(
            image_2d, (IMG_SZ, IMG_SZ), method='nearest'
        )

    image_patches = extract_patches(
        image_2d,
        sizes=[1, IMG_SZ, IMG_SZ, 1],
        strides=[1, PATCH_STEP, PATCH_STEP, 1],
        rates=[1, 1, 1, 1],
        padding='VALID'
    )

    return tf.reshape(
        image_patches,
        [image_patches.shape[1] * image_patches.shape[2], IMG_SZ, IMG_SZ, 1]
    )

# Based on: https://github.com/MinaJf/FU-net/blob/HEAD/image_loder.py
def _pre_process_img(img):
    img = tf.cast(img, tf.float32)

    # standardization (zero mean)
    img -= tf.math.reduce_mean(img)
    img /= tf.math.reduce_std(img)

    # normalize between [0, 1]
    img -= tf.math.reduce_min(img)
    img /= tf.math.reduce_max(img)

    return img

# Carregando dados

Imports da seção

In [ ]:
from random import randint

import matplotlib.pyplot as plt
from numpy import unique
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.utils import to_categorical

In [ ]:
def to_dataset(data, labels):
    dataset = tf.data.Dataset.from_tensor_slices((data, labels))
    dataset = dataset.shuffle(
        buffer_size=DT_BUFFER_SZ,
        seed=RANDOM_SEED,
        reshuffle_each_iteration=True
    )
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.repeat()
    dataset = dataset.prefetch(2)

    return dataset

In [ ]:
training_paths = file_paths(TRAINING_DIR)

In [ ]:
print(len(training_paths['config']))
print(len(training_paths['ground_truth']))
print(len(training_paths['images']))
print(len(training_paths['4d']))

In [ ]:
raw_train_data, raw_train_labels = load_data(training_paths)

In [ ]:
print(f'raw_train_data: {raw_train_data.shape}\n\t{raw_train_data.dtype}')
print(f'raw_train_labels: {raw_train_labels.shape}\n\t{raw_train_labels.dtype}')

In [ ]:
classes_weight = compute_class_weight(
    'balanced',
    unique(raw_train_labels),
    tf.reshape(raw_train_labels, tf.size(raw_train_labels)).numpy()
)

In [ ]:
print(f'classes_weight: {len(classes_weight)}\n{classes_weight}')

In [ ]:
raw_train_labels = to_categorical(raw_train_labels, CLASSES_CNT, dtype='int32')

In [ ]:
train_data, test_data, train_labels, test_labels = train_test_split(
    raw_train_data.numpy(), raw_train_labels,
    test_size=VALIDATION_PROPORTION,
    random_state=RANDOM_SEED
)

In [ ]:
print(f'test_data: {test_data.shape}')
print(f'test_labels: {test_labels.shape}')

In [ ]:
train_data, xval_data, train_labels, xval_labels = train_test_split(
    train_data, train_labels,
    test_size=XVALIDATION_PROPORTION,
    random_state=RANDOM_SEED
)

In [ ]:
print(f'train_data: {train_data.shape}')
print(f'xval_data: {xval_data.shape}')
print(f'train_labels: {train_labels.shape}')
print(f'xval_labels: {xval_labels.shape}')

In [ ]:
train_dataset = to_dataset(train_data, train_labels)

In [ ]:
xval_dataset = to_dataset(xval_data, xval_labels)

In [ ]:
test_dataset = to_dataset(test_data, test_labels)

Pré-visualização

In [ ]:
rnd_idx = randint(0, len(train_data))

fig, axes = plt.subplots(1, 2)

train_img = tf.squeeze(train_data[rnd_idx], axis=2)
train_gt = tf.argmax(train_labels[rnd_idx], axis=2)

axes[0].imshow(train_img, cmap='hot')
axes[1].imshow(train_gt, cmap='hot')

plt.show()